# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

In [ ]:
import json
import os
import glob
from tqdm import tqdm

atlas_data_folder = os.getenv('ATLAS_DATA_FOLDER')

data_folder = os.path.join(atlas_data_folder, 'mouse', 'v3')
f = open(os.path.join(data_folder, 'brain_regions.json'))
obj = json.load(f)

region_colors=dict()
def node_color(node):
    node_id = node['id']
    color = node['color_hex_triplet']
    region_colors[node_id] = color
    for child in node['children']:
        node_color(child)

for node in obj['msg']:
    node_color(node)

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))



In [ ]:
def add_material_to_obj(obj_file, material_name, diffuse_color):
    # Open the OBJ file in read mode
    with open(obj_file, 'r') as file:
        lines = file.readlines()

    # Insert the material definition before the first usemtl line
    lines.insert(0, 'mtllib {}\n'.format(os.path.basename(obj_file).replace('.obj', '.mtl')))

    # Write the modified lines back to the OBJ file
    with open(obj_file, 'w') as file:
        file.writelines(lines)

    # Create the material file with diffuse color
    mtl_filename = obj_file.replace('.obj', '.mtl')
    with open('{}'.format(mtl_filename), 'w') as mtl_file:
        mtl_file.write('newmtl {}\n'.format(material_name))
        mtl_file.write('Kd {} {} {}\n'.format(*[int(diffuse_color[i:i+2], 16)/255 for i in (1, 3, 5)]))
        # You might want to add more properties like specular color, shininess, etc. if needed

    # print("Material '{}' added to '{}' with diffuse color '{}' successfully.".format(material_name, obj_file, diffuse_color))


# Example usage:
mesh_folder = os.path.join(data_folder, 'meshes', 'obj')
mesh_files = glob.glob(mesh_folder + '/*.obj')
for mesh_file in tqdm(mesh_files):
    base_name = os.path.basename(mesh_file).split('.')[0]
    color = region_colors[int(base_name)]
    add_material_to_obj(mesh_file, 'default', color)
